In [5]:
from PyPDF2 import PdfReader
import langchain
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import dotenv 
import warnings
warnings.filterwarnings("ignore")

In [26]:
# Load the environment variables from the .env file
dotenv.load_dotenv()

True

In [27]:
# Access the environment variables just like you would with os.environ
key = os.getenv("OPENAI_API_KEY")

In [8]:
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()

    return text

In [17]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size =10000,
        chunk_overlap =1000
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [20]:
def get_vector_store(text_chunks):
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(texts=text_chunks,embedding=embeddings)
    vector_store.save_local("faiss_index")
    

In [21]:
def get_conversional_chain():
    
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
    model = ChatOpenAI(
        openai_api_key = key,
        model_name = "gpt-3.5-turbo",
        temperature = 0.3
    )
    prompt = PromptTemplate(
        template = prompt_template,
        input_variables=["context","question"]
    )
    chain = load_qa_chain(
        model,
        chain_type="stuff",
        prompt=prompt
    )
    return chain


In [24]:
def user_input(user_question):
    embeddings = OpenAIEmbeddings()

    new_db = FAISS.load_local("faiss_index",embeddings)
    docs = new_db.similarity_search(user_question)

    chain = get_conversional_chain()

    response = chain(
        {"input_documents":docs,"question":user_question},
        return_only_outputs=True
    )
    print(response)
    #st.write("Reply: ", response["output_text"])